In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [10]:
terms = {
        'BonusMalus': {'type': 'ns'}, 
        'VehAge': {'type': 'ns'},
        'DrivAge': {'type': 'ns'}, 
        'VehPower': {'type': 'ns'}, 
        'Area': {'type': 'target_encoding'},
        'Region': {'type': 'target_encoding'},
        'VehGas': {'type': 'target_encoding'},
        'VehBrand': {'type': 'target_encoding'}
    }

interactions=[{
        "VehPower": {"type": "linear"},
        "Region": {"type": "categorical"},
        "target_encoding": True,
        "include_main": False
    }]

result = rs.glm_dict(
    response='ClaimCount',
    terms=terms,
    interactions=interactions,
    data=data,
    family='poisson',
    offset ='Exposure'
).fit()

In [11]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ ns(BonusMalus, df=4) + ns(VehAge, df=4) + ns(DrivAge, df=4) + ns(VehPower, df=4) + TE(Area) + TE(Region) + TE(VehGas) + TE(VehBrand)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 42, 'df_resid': 406397, 'converged': True, 'iterations': 7, 'regularization': {'type': 'ridge', 'alpha': 0.409756, 'l1_ratio': 0.0}}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21278.79, loss=0.309461, deviance=125776.9, log_likelihood=-83662.28, aic=167408.55, gini=0.2846, auc=0.6423, ae_ratio=1.0061, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1925.63, actual=0.393119, predicted=0.075256, ae_ratio=5.2237), DecileMetrics(decile=2, n=40643, exposure=4248.53, actual=0.222194, predicted=0.088556, ae_ratio=2.5091), DecileMetrics(decile=3, n=40643, exposure=8292.64, actual=0.162313, predicted=0.084462, ae_ratio=1.9217), 

In [12]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        678012
Link Function:       (default)       Df Residuals:            677970
Method:              IRLS + Ridge    Df Model:                    41
Scale:               1.0000          Alpha (λ):               0.4098
L1 Ratio:            0.00            Iterations:                   7
Non-zero coefs:      41             

Log-Likelihood:         -139094.8792 Deviance:                209189.6123
AIC:                     278273.7584 Null Deviance:           220244.0296
BIC:                     278753.6891 Pearson chi2:             1521072.62
Converged:           True           

------------------------------------------------------------------------------
Variable                  Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept              -2.3465 

In [13]:
result.predict(holdout_data)

array([0.05035605, 0.0466738 , 0.0485924 , ..., 0.00518024, 0.01665991,
       0.07062843], shape=(135957,))

In [6]:
# Model spec before interaction
terms = {
    "BonusMalus": {"type": "linear"},
    "VehAge": {"type": "ns", "df": 5},
    "DrivAge": {"type": "linear"},
    "Region": {"type": "target_encoding"},
    "VehBrand": {"type": "target_encoding"},
    "Area": {"type": "target_encoding"},
}
# Interaction to add (DrivAge x BonusMalus)
interactions = [
    {
        "DrivAge": {"type": "linear"},
        "Region": {"type": "target_encoding"},
        "include_main": False,
    }
]
# Fit with interaction
result = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit()


In [7]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406428
Method:              IRLS            Df Model:                    10
Scale:               1.0000          Iterations:                   6

Log-Likelihood:          -84126.9808 Deviance:                126706.3119
AIC:                     168275.9616 Null Deviance:           132328.3090
BIC:                     168396.0287 Pearson chi2:              935253.45
Converged:           True           

------------------------------------------------------------------------------
Variable                 Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept             -4.3712     0.1587  -27.542  <0.0001   [ -4.6823,  -4.0602]  ***
BonusMalus             0.0220     0.0004   54.664  

In [8]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [14]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ ns(BonusMalus, df=4) + ns(VehAge, df=4) + ns(DrivAge, df=4) + ns(VehPower, df=4) + TE(Area) + TE(Region) + TE(VehGas) + TE(VehBrand)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 42, 'df_resid': 406397, 'converged': True, 'iterations': 7, 'regularization': {'type': 'ridge', 'alpha': 0.409756, 'l1_ratio': 0.0}}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21278.79, loss=0.309461, deviance=125776.9, log_likelihood=-83662.28, aic=167408.55, gini=0.2846, auc=0.6423, ae_ratio=1.0061, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1925.63, actual=0.393119, predicted=0.075256, ae_ratio=5.2237), DecileMetrics(decile=2, n=40643, exposure=4248.53, actual=0.222194, predicted=0.088556, ae_ratio=2.5091), DecileMetrics(decile=3, n=40643, exposure=8292.64, actual=0.162313, predicted=0.084462, ae_ratio=1.9217), 

In [ ]:
predictions = result.predict(holdout_data)

In [ ]:
model_bytes = result.to_bytes()
with open("model.bin", "wb") as f:
    f.write(model_bytes)

In [ ]:
with open("model.bin", "rb") as f:
    loaded = rs.GLMModel.from_bytes(f.read())

In [ ]:
predictions2 = loaded.predict(holdout_data)

In [ ]:
import numpy as np
print(np.array_equal(predictions2, predictions))

True


In [ ]:
#writes analysis/diagnostics
loaded.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

In [ ]:
result2 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "Region": {"type": "target_encoding"},
        "BonusMalus": {"type": "linear"},
        "BonusMalus2": {"type": "expression", "expr": "BonusMalus ** 2"}
    },
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net"
)

In [ ]:
#writes analysis/diagnostics
result2.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

In [ ]:
result3 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "BonusMalus": {"type": "ns"},
        "VehAge": {"type": "ns"},
        "DrivAge": {"type": "linear"},
        "VehBrand": {"type": "target_encoding"}
    },
    interactions= [
        {
            "VehAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        },
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ],
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net",
    include_unregularized=False
)

In [ ]:
import rustystats as rs

# Model spec that caused the issue when elastic_net was applied
terms = {
    "BonusMalus": {'type' : "linear"},
    "VehAge": {'type' : "ns"}, 
    "DrivAge": {'type' : "ns"},
    "Area": {'type' : "target_encoding"},
}

interactions= [
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ]

# Build the model
glm = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,  # freMTPL2freq dataset
    family="poisson",
    offset="Exposure",  # log-linked
)

# This is where the issue occurs - regularized fit + diagnostics
result = glm.fit()

# Check if diagnostics returns None for deviance
diag = result.diagnostics(train_data=train_data, test_data=holdout_data)
print(diag.to_dict())  # Check train_test.test.deviance

{'model_summary': {'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=4) + ns(DrivAge, df=4) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 22, 'df_resid': 406417, 'converged': True, 'iterations': 7, 'regularization': {'type': 'ridge', 'alpha': 0.4, 'l1_ratio': 0.0}}, 'train_test': {'train': {'dataset': 'train', 'n_obs': 406439, 'total_exposure': 214930.9, 'total_actual': 21408.0, 'total_predicted': 21530.19, 'loss': 0.310655, 'deviance': 126262.46, 'log_likelihood': -83905.05, 'aic': 167854.11, 'gini': 0.2747, 'auc': 0.6374, 'ae_ratio': 0.9943, 'ae_by_decile': [{'decile': 1, 'n': 40643, 'exposure': 1909.44, 'actual': 0.392262, 'predicted': 0.077322, 'ae_ratio': 5.0731}, {'decile': 2, 'n': 40643, 'exposure': 4144.57, 'actual': 0.234765, 'predicted': 0.091204, 'ae_ratio': 2.5741}, {'decile': 3, 'n': 40643, 'exposure': 8115.13, 'actual': 0.167711, 'predicted': 0.087742, 'ae_ratio': 1.9114}, {'decile': 4, 'n': 40643, 'exposure': 13627.33, 'actual': 0.125263